# EDA_3 - Missing country_codes replacement from the full datasets

In [ ]:
# Initially, the 10 critical working files lacked country_codes. To address this, a script was developed to merge these -
# 10 files into a unified array using an append method. Following this, the script generates pairs from the customer - 
# identifier and event timestamp, and then attempts to find corresponding matches for these pairs across the comprehensive - 
# 28-million-row database. The goal is to ascertain if country_codes are present in other event types through these matches. 
# Should a match be found, the relevant country_code is to be inserted into the newly appended table.

In [ ]:
import os
import pandas as pd
import numpy as np
from pathlib import Path

# Define paths
source_dir = r"C:\Users\user\Desktop\.....\PY_10_PYClean_Source_V6"
output_dir = r"C:\Users\user\Desktop\.....\PY_10_PYClean_Source_V7"
full_db_path = r"C:/Users/user/Desktop/...../teleprompter_hashed_fixed_v3.csv"

# List of input files
input_files = [
    "billing_issue_event", "cancellation_event", "expiration_event", "initial_purchase_event",
    "product_change_event", "renewal_event", "trial_cancelled_event", "trial_converted_event",
    "trial_started_event", "uncancellation_event"
]

# Read and append all CSVs
dfs = []
for file in input_files:
    file_path = os.path.join(source_dir, f"{file}.csv")
    df = pd.read_csv(file_path)
    dfs.append(df)
appended_df = pd.concat(dfs, ignore_index=True)
print("✅ 10 tábla összeappendelve")

# Read full database with correct separator
full_db_df = pd.read_csv(full_db_path, sep=';')
print(f"✅ Eredeti adatbázis betöltve, oszlopok: {list(full_db_df.columns)}")

# Clean distinct_id to keep only numbers
full_db_df['distinct_id'] = full_db_df['distinct_id'].astype(str).str.extract('(\d+)', expand=False)

# Filter known country_code
known_country_df = full_db_df[full_db_df['country_code'] != 'unknown'][['distinct_id', 'event_date', 'country_code']]

# Ensure correct types for merging
appended_df['distinct_id'] = appended_df['distinct_id'].astype(str)
known_country_df['distinct_id'] = known_country_df['distinct_id'].astype(str)

# Merge on distinct_id + event_date
merged_df = pd.merge(
    appended_df,
    known_country_df,
    on=['distinct_id', 'event_date'],
    how='left',
    suffixes=('', '_new')
)

# Fill 'unknown' country_code with found values
merged_df['country_code'] = np.where(
    merged_df['country_code'] == 'unknown',
    merged_df['country_code_new'],
    merged_df['country_code']
)

# Drop the helper column
merged_df = merged_df.drop(columns=['country_code_new'])

# Save split files back to V7 folder
for file in input_files:
    output_path = os.path.join(output_dir, f"{file}.csv")
    df_part = merged_df[merged_df['event'] == file]
    df_part.to_csv(output_path, index=False)
    print(f"✅ {file} elmentve ide: {output_path}")

print("🎉 Ok!")